# Inclusão de Bibliotecas

In [1]:
from os import listdir # Biblioteca responsável por capturar informações de arquivos dentro de um diretório.

import os

import numpy as np # Cálculos numéricos

import pandas as pd # Criação e manipulação de dataframes.

import seaborn as sns # Biblioteca para plotar diversos tipos de gráficos.

import matplotlib.pyplot as plt # Biblitoeca para plotar gráficos.

import wave # Biblioteca com métodos que permitam converter dados raw em .wav

import struct # Biblioteca responsável por trabalhar com dados raw.

import librosa # A python package for music and audio analysis.

import librosa.display # librosa package visualization.

import random # Random functions.

# Métricas de erro para validação do modelo gerado. 
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report, confusion_matrix

# Aplicação de PCA no dataset.
from sklearn.decomposition import PCA






pd.set_option('display.max_rows', None) # Mostra todas as linhas do dataframe.
pd.set_option('display.max_columns', None) # Mostra todas as colunas do dataframe.


In [2]:
%matplotlib inline

# Funções para Criação do Dataframe

In [16]:
####### Funções necessárias para gerar o dataset conforme o status da marcha e profile_id: ###########

# Retorna todos os arquivos com extensão .csv em uma lista. Os elementos são ordenados em ordem crescente.
def findCSV_filename(path_to_dir, suffix = ".csv"):
    
    # Lista os arquivos no diretório path_to_dir 
    filenames = listdir(path_to_dir) 
    
    # Retorna uma lista com os arquivos com final .csv em ordem crescente baseado no nome do arquivo.
    return sorted([filename for filename in filenames if filename.endswith( suffix )])

# Separa todos os arquivos .csv em listas diferentes conforme a categoria da marcha.
def FormatScreeningFilenames(filenames):
    
    MS_filenames = [] # Lista da categoria Marcha Soldado"
    MN_filenames = [] # Lsta da categoria "Marcha Normal"
    MM_filenames = [] # Lista da categoria "Marcha Mancando"
    MD_filenames = [] # Lista da categoria "Marcha a Definir"

    
    # Lista os arquivos no diretório path_to_dir
    for filename in filenames:
        
        if (filename.find("MS", 4, 6) != -1): # Se encontrar "MS" no nome do arquivo:
            MS_filenames.append(filename) # Gera lista com todos os "MS" .csv
            
            # Incrementa numeração de acordo com o número do último arquivo
            lastMS_fileNumber = int(findCSV_filename("/home/trs/Gait_Project/GaitDataset/Triagem/MS")[-1][:4])+1
            
            
        elif (filename.find("MN", 4, 6) != -1): # Se encontrar "MN" no nome do arquivo:
            MN_filenames.append(filename) # Gera lista com todos os "MN" .csv
            
            # Incrementa numeração de acordo com o número do último arquivo na pasta MN
            lastMN_fileNumber = int(findCSV_filename("/home/trs/Gait_Project/GaitDataset/Triagem/MN")[-1][:4])+1
            
        elif (filename.find("MM", 4, 6) != -1): # Se encontrar "MM" no nome do arquivo:
            MM_filenames.append(filename) # Gera lista com todos os "MM" .csv
            
            # Incrementa numeração de acordo com o número do último arquivo na pasta MM
            lastMM_fileNumber = int(findCSV_filename("/home/trs/Gait_Project/GaitDataset/Triagem/MM")[-1][:4])+1
             
        else: # Se encontrar "MD" no nome do arquivo:
            MD_filenames.append(filename) # Gera lista com todos os "MD" .csv
            
            # Incrementa numeração de acordo com o número do último arquivo na pasta MD
            lastMD_fileNumber = int(findCSV_filename("/home/trs/Gait_Project/GaitDataset/Triagem/MD")[-1][:4])+1
            
    

    for filename in MS_filenames: 
        
        # Move os arquivos MM para a pasta de destino MS.
        os.rename("/home/trs/Gait_Project/GaitDataset/Triagem/{}".format(filename),
                  "/home/trs/Gait_Project/GaitDataset/Triagem/{}".format('{:0>4}'.format(lastMS_fileNumber)
                                                                       +filename[4:]))
        lastMS_fileNumber+=1 
        
    
    for filename in MM_filenames:
        
        # Move os arquivos MM para a pasta de destino MM.
        os.replace("/home/trs/Gait_Project/GaitDataset/Triagem/{}".format(filename),
                  "/home/trs/Gait_Project/GaitDataset/Triagem/MM/{}".format('{:0>4}'.format(lastMM_fileNumber)
                                                                       +filename[4:]))
        lastMM_fileNumber+=1
    
    for filename in MN_filenames:
        
        # Move os arquivos MM para a pasta de destino MN.
        os.replace("/home/trs/Gait_Project/GaitDataset/Triagem/{}".format(filename),
                  "/home/trs/Gait_Project/GaitDataset/Triagem/MN/{}".format('{:0>4}'.format(lastMN_fileNumber)
                                                                       +filename[4:]))
        lastMN_fileNumber+=1
    
            
        

# Retorna uma lista que contém o profile_id de todos os arquivos com extensão .csv
def getProfile_id(filenames):
    
    profile_id=[]
    
    for item in filenames: # Para cada arquivo na lista filenames,
        
        for i in range(500): # Dentro de cada arquivo, faça 500 vezes:
            
            profile_id.append(item[:4]) #Pega os 4 primeiros caracteres e armazena na lista profile_id.
            
    return profile_id 

# Retorna uma lista com o status da marcha de cada profile_id diferente.     
def getGait_status(filenames):
    
    gaitStatus=[]

    for item in filenames: # Para cada arquivo na lista filenames,
        
        for i in range(500): # Dentro de cada arquivo, faça 500 vezes:
            
            gaitStatus.append(item[4:6]) # Pega o quinto caractere e armazena na lista gaitStatus.
            
    return gaitStatus

# Cria um dataframe vertical com todos os arquivos .csv dados na lista filenames.
def createDataframe(filenames):

    i = 1
    
    df = pd.read_csv(filenames[0]) # Inicializa o dataframe com o primeiro arquivo .csv da lista filenames.
    
    while i < len(filenames): # Percorre toda a lista filenames.
        
        df_temporary = pd.read_csv(filenames[i]) # Cria um dataframe temporario.
        
        df = pd.concat([df,df_temporary]).reset_index(drop=True) # Concatena no dataframe o dataframe temporário. 
        
        i += 1
        
    return(df)

# Insere as colunas 'Gait_Status' e 'Profile_ID' no dataframe vertical.
def insertColumns(df, gaitStatus, profile_id):
    
    df['Gait_Status'] = gaitStatus # Passa todos os elementos, com respectiva ordem, para a coluna especificada.
    
    df['Profile_ID'] = profile_id  # Passa todos os elementos, com respectiva ordem, para a coluna especificada.
    
    return df

# Cria um Dataframe horizontal com base no dataframe vertical.
def createHorizontalDataframe(df, filenames):
    
    profile_id=[]
    
    for item in filenames: # Para cada item na lista filenames: 
        profile_id.append(item[:4]) # Pega os 4 primeiros caracteres e armazena numa lista profile_id.

    gaitStatus=[]
    for item in filenames: # Para cada item na lista filenames:
        gaitStatus.append(item[4:6]) # Pega o quinto caractere e armazena numa lista gaitStatus


    xac = []
    for i in range(500): # Faça 500 vezes: 
        xac.append("Xac{:0>3}".format(i)) # xac = [ Xac000, Xac001, ... , Xacc499]
    yac = []
    for i in range(500): # Faça 500 vezes: 
        yac.append("Yac{:0>3}".format(i)) # yac = [ Yac000, Yac001, ... , Yacc499]
    zac = []
    for i in range(500):
        zac.append("Zac{:0>3}".format(i))
        
    xang = []
    for i in range(500):
        xang.append("RotX{:0>3}".format(i))
        
    yang = []
    for i in range(500):
        yang.append("RotY{:0>3}".format(i))
        
    zang = []
    for i in range(500):
        zang.append("RotZ{:0>3}".format(i))
        
    tempo = []
    for i in range(500):
        tempo.append("Tempo{:0>3}".format(i))
        
    columns_label = [xac,yac,zac,xang,yang,zang,tempo]
    
    columns_df = []
    
    columns_df.append('profile_ID') # Acrescenta a coluna 'profile_ID'
    columns_df.append('Gait Status') # Acrescenta a coluna 'Gait Status'

    for item in columns_label: # Para cada coluna,
        for i in range(500): # Faça 500 vezes
            columns_df.append(item[i]) # Ao final, temos uma lista com todos as colunas rotuladas adequadamente.

    columns_labelstring = ['xac','yac','zac','xang','yang','zang','tempo']
    
    
    

    df_horizontal = pd.DataFrame(columns = columns_df) # Cria as colunas do dataframe com a correta rotulação.
    
    count_concat = 0
    
    j = 0
    
    k = 500
    while count_concat < len(filenames): # Enquanto não concatenar todos os arquivos da lista filenames,
        vetorzao = []
        vetorzao.append(profile_id[count_concat]) # Pega o ID dentro da lista profile_id.
        
        vetorzao.append(gaitStatus[count_concat]) # Pega o estado da marcha dentro da lista gaitStatus.

        for s in range(len(columns_labelstring)): # Percorre todo a lista columns_labelstring.
            
            for i in range(j, k): 
                vetorzao.append(dataframe['{}'.format(columns_labelstring[s])][i])    
        df_horizontalbuffer = pd.DataFrame([vetorzao], columns = columns_df)
        df_horizontal = pd.concat([df_horizontal,df_horizontalbuffer]).reset_index(drop=True)
        j += 500
        k += 500
        count_concat += 1
    return df_horizontal


In [48]:
filenames = findCSV_filename("/home/trs/Gait_Project/GaitDataset/Triagem")
FormatScreeningFilenames(filenames)

        
        


# Criação do Dataframe Vertical

In [25]:
filenames = findCSV_filename("/home/trs/Gait_Project/GaitDataset/Triagem/full_Dataset")

gaitStatus = getGait_status(filenames)
profile_id = getProfile_id(filenames)

filenames[0]
df = pd.read_csv(filenames[0]) # Inicializa o dataframe com o primeiro arquivo .csv da lista filenames.

FileNotFoundError: [Errno 2] File 0001MM2986RS186.csv does not exist: '0001MM2986RS186.csv'

In [22]:
filenames = findCSV_filename("/home/trs/Gait_Project/GaitDataset/Triagem/full_Dataset")

gaitStatus = getGait_status(filenames)

profile_id = getProfile_id(filenames)

df = createDataframe(filenames)



FileNotFoundError: [Errno 2] File 0001MM2986RS186.csv does not exist: '0001MM2986RS186.csv'

In [7]:
df.shape

NameError: name 'df' is not defined

In [8]:
dataframe = insertColumns(df,gaitStatus,profile_id)


In [9]:
dataframe.head()

,xac,yac,zac,xang,yang,zang,tempo,amostra,Gait_Status,Profile_ID
0,-3581,2242,5342,182,1150,-871,1,0,N,0001
1,-4655,2228,5446,59,774,-882,4075,1,N,0001
2,-3121,1979,5965,-67,261,-919,13887,2,N,0001
3,-1726,2095,5258,-111,1,-955,24028,3,N,0001
4,-216,1982,5026,-122,-54,-907,33844,4,N,0001


In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89000 entries, 0 to 88999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   xac          89000 non-null  int64 
 1   yac          89000 non-null  int64 
 2   zac          89000 non-null  int64 
 3   xang         89000 non-null  int64 
 4   yang         89000 non-null  int64 
 5   zang         89000 non-null  int64 
 6   tempo        89000 non-null  int64 
 7   amostra      89000 non-null  int64 
 8   Gait_Status  89000 non-null  object
 9   Profile_ID   89000 non-null  object
dtypes: int64(8), object(2)
memory usage: 6.8+ MB


# Criação do Dataframe Horizontal

In [11]:
df_horizontal = createHorizontalDataframe(dataframe, filenames)

In [12]:
df_horizontal.head()

profile_ID Gait Status Xac000 Xac001 Xac002 Xac003 Xac004 Xac005 Xac006  \
0       0001           N  -3581  -4655  -3121  -1726   -216   1781   1607   
1       0002           N    264   5223   7610   3760   1422   1083    729   
2       0003           N   -870   -769   -253    227      2     15   -400   
3       0004           N   1634   1951   2401   1388    577    386   1166   
4       0005           N   2419  -3198  -5572  -3938  -4115  -3154  -1565   

  Xac007 Xac008 Xac009 Xac010 Xac011 Xac012 Xac013 Xac014 Xac015 Xac016  \
0   -273  -1382  -1475  -1618  -1746  -1859  -1791  -1672  -1342   -949   
1   -686  -1194  -2333  -1982   -972   -315    436   1119   1515   2283   
2   -520   -568   -384   -139     71     63    308    302    566    966   
3   1811   2076   1899   1276   1408   1345   1107    946   1231   1291   
4    892    778   -682  -1737  -2414  -2819  -2562  -2092  -2554  -2775   

  Xac017 Xac018 Xac019 Xac020 Xac021 Xac022 Xac023 Xac024 Xac025 Xac026  \
0   -935   -885   -288   -606     72   1135    714    991   1497   1554   
1   2367   2309   1878   1583   1945   3393   4661   5485   5632   5366   
2   1521   1070   1725   2231   1796   2409   2627   2368   2297   2499   
3   1668   1705   1753   1849   2150   2169   2242   2173   1952   1760   
4  -2993  -3093  -2822  -2752  -2856  -2588  -1899  -1460  -1243   -929   

  Xac027 Xac028 Xac029 Xac030 Xac031 Xac032 Xac033 Xac034 Xac035 Xac036  \
0   2632   2448   2319   2289   2145   2083   2292   1997   1550   1747   
1   5207   5794   7047   7180   7404   8474  10112  12364  14802  15772   
2   2205   2347   2019   1715   1432   1299   1258   1222   1366   1332   
3   2057   2453   1885   1996   1902   1733   1270   1009   1055   1121   
4   -530   -229   -529    749    939   1543   1944   1867   1376   1135   

  Xac037 Xac038 Xac039 Xac040 Xac041 Xac042 Xac043 Xac044 Xac045  Xac046  \
0   1686   1764   1643   1218   1228   1450   1729   1765   2302    2233   
1  16350  19127  24855  30946  30894  24159  17890  14852   6456  -13024   
2   1821   2920   3836   3545   2057   3113   2444    371   -629    -510   
3   1263   1760   2716   3245   2984   3093   2619    889    290    -146   
4   1050   1029   1074    885    741    718    415    531    358     347   

  Xac047 Xac048 Xac049  Xac050 Xac051 Xac052 Xac053 Xac054 Xac055 Xac056  \
0   1916   2128   2795    2686   1743   2024   1860   -668  -2069  -3099   
1  30257  30306   9019   27775   6133   6599   5973     34   -931   5121   
2   -937  -2226  -3962   -5294  -4063   2183  12087   9140   4755   3553   
3  -1822  -3402  -6456  -10012  -6847   2973  12507   6460   4409   5911   
4    422     13     99     974   1507   2195   2084    618    915   1800   

  Xac057 Xac058 Xac059 Xac060 Xac061 Xac062 Xac063 Xac064 Xac065 Xac066  \
0  -5204  -9375  -9698  -4032   8704  10506   6388   4405   1555    102   
1   8565   1794  -3591  -6217  -3841   -641    -68   1157   1279   -461   
2   1866  -1085  -2116  -3143  -4841  -4463    152   7001   5694   1480   
3   2950  -1476  -2139  -1207  -1586    559   6600   6917   1824    422   
4    504   -730  -1648  -2896  -4688  -7196  -8486  -7070  -2195   7785   

  Xac067 Xac068 Xac069 Xac070 Xac071 Xac072 Xac073 Xac074 Xac075 Xac076  \
0    832    928    749   3690   8169   3806   1072    595    322   -839   
1  -1708  -2288  -2407  -2831  -2993  -1838  -2142  -1861  -1838   -860   
2    240    901    745    364  -2727  -4688  -5806  -5703  -2386   -264   
3   2038   3207   3584   2908   2574   2273    -71  -1999  -4061  -3955   
4   9199   3711   3674   3481    864   -676   -869   -993  -1058    798   

  Xac077 Xac078 Xac079 Xac080 Xac081 Xac082 Xac083 Xac084 Xac085 Xac086  \
0  -1494  -2619  -2728  -1505   -780   -199    374    908   1080   1019   
1   -242    214    310     80    297    -77    886   1521   1872   2428   
2    268    975   1225   1492   1319   1498   2040   4042   5879   6430   
3  -1605   -656    179   1327   1231   1136   2224   2922   4510   523

In [13]:
# Transformar a feature categórica 'Gait Status' em resposta binária. Normal = 1 -> I = 0 e vice-versa.
df_dummies= df_horizontal
df_dummies = pd.get_dummies(df_dummies, columns= ['Gait Status'], prefix = 'Gait_Status', drop_first= True)
df_dummies.head()


profile_ID Xac000 Xac001 Xac002 Xac003 Xac004 Xac005 Xac006 Xac007 Xac008  \
0       0001  -3581  -4655  -3121  -1726   -216   1781   1607   -273  -1382   
1       0002    264   5223   7610   3760   1422   1083    729   -686  -1194   
2       0003   -870   -769   -253    227      2     15   -400   -520   -568   
3       0004   1634   1951   2401   1388    577    386   1166   1811   2076   
4       0005   2419  -3198  -5572  -3938  -4115  -3154  -1565    892    778   

  Xac009 Xac010 Xac011 Xac012 Xac013 Xac014 Xac015 Xac016 Xac017 Xac018  \
0  -1475  -1618  -1746  -1859  -1791  -1672  -1342   -949   -935   -885   
1  -2333  -1982   -972   -315    436   1119   1515   2283   2367   2309   
2   -384   -139     71     63    308    302    566    966   1521   1070   
3   1899   1276   1408   1345   1107    946   1231   1291   1668   1705   
4   -682  -1737  -2414  -2819  -2562  -2092  -2554  -2775  -2993  -3093   

  Xac019 Xac020 Xac021 Xac022 Xac023 Xac024 Xac025 Xac026 Xac027 Xac028  \
0   -288   -606     72   1135    714    991   1497   1554   2632   2448   
1   1878   1583   1945   3393   4661   5485   5632   5366   5207   5794   
2   1725   2231   1796   2409   2627   2368   2297   2499   2205   2347   
3   1753   1849   2150   2169   2242   2173   1952   1760   2057   2453   
4  -2822  -2752  -2856  -2588  -1899  -1460  -1243   -929   -530   -229   

  Xac029 Xac030 Xac031 Xac032 Xac033 Xac034 Xac035 Xac036 Xac037 Xac038  \
0   2319   2289   2145   2083   2292   1997   1550   1747   1686   1764   
1   7047   7180   7404   8474  10112  12364  14802  15772  16350  19127   
2   2019   1715   1432   1299   1258   1222   1366   1332   1821   2920   
3   1885   1996   1902   1733   1270   1009   1055   1121   1263   1760   
4   -529    749    939   1543   1944   1867   1376   1135   1050   1029   

  Xac039 Xac040 Xac041 Xac042 Xac043 Xac044 Xac045  Xac046 Xac047 Xac048  \
0   1643   1218   1228   1450   1729   1765   2302    2233   1916   2128   
1  24855  30946  30894  24159  17890  14852   6456  -13024  30257  30306   
2   3836   3545   2057   3113   2444    371   -629    -510   -937  -2226   
3   2716   3245   2984   3093   2619    889    290    -146  -1822  -3402   
4   1074    885    741    718    415    531    358     347    422     13   

  Xac049  Xac050 Xac051 Xac052 Xac053 Xac054 Xac055 Xac056 Xac057 Xac058  \
0   2795    2686   1743   2024   1860   -668  -2069  -3099  -5204  -9375   
1   9019   27775   6133   6599   5973     34   -931   5121   8565   1794   
2  -3962   -5294  -4063   2183  12087   9140   4755   3553   1866  -1085   
3  -6456  -10012  -6847   2973  12507   6460   4409   5911   2950  -1476   
4     99     974   1507   2195   2084    618    915   1800    504   -730   

  Xac059 Xac060 Xac061 Xac062 Xac063 Xac064 Xac065 Xac066 Xac067 Xac068  \
0  -9698  -4032   8704  10506   6388   4405   1555    102    832    928   
1  -3591  -6217  -3841   -641    -68   1157   1279   -461  -1708  -2288   
2  -2116  -3143  -4841  -4463    152   7001   5694   1480    240    901   
3  -2139  -1207  -1586    559   6600   6917   1824    422   2038   3207   
4  -1648  -2896  -4688  -7196  -8486  -7070  -2195   7785   9199   3711   

  Xac069 Xac070 Xac071 Xac072 Xac073 Xac074 Xac075 Xac076 Xac077 Xac078  \
0    749   3690   8169   3806   1072    595    322   -839  -1494  -2619   
1  -2407  -2831  -2993  -1838  -2142  -1861  -1838   -860   -242    214   
2    745    364  -2727  -4688  -5806  -5703  -2386   -264    268    975   
3   3584   2908   2574   2273    -71  -1999  -4061  -3955  -1605   -656   
4   3674   3481    864   -676   -869   -993  -1058    798   5380   5590   

  Xac079 Xac080 Xac081 Xac082 Xac083 Xac084 Xac085 Xac086 Xac087 Xac088  \
0  -2728  -1505   -780   -199    374    908   1080   1019   1141   1556   
1    310     80    297    -77    886   1521   1872   2428   1974   2030   
2   1225   1492   1319   1498   2040   4042   5879   6430   6844   7482   
3    179   1327   1231   1136   2224   2922   4510   5238 

In [14]:
tempo_drop = df_dummies.loc[:,'Tempo000':'Tempo499']
df_pca = df_dummies.drop(['profile_ID', 'Gait_Status_N'], axis = 1) # Elimina as feature 'profile_ID' e Gait_Status_N
df_pca = df_pca.drop(tempo_drop, axis = 1) # Elimina as features de step time.
df_pca.head()  # df_pca contém apenas os atributos agora



Xac000 Xac001 Xac002 Xac003 Xac004 Xac005 Xac006 Xac007 Xac008 Xac009  \
0  -3581  -4655  -3121  -1726   -216   1781   1607   -273  -1382  -1475   
1    264   5223   7610   3760   1422   1083    729   -686  -1194  -2333   
2   -870   -769   -253    227      2     15   -400   -520   -568   -384   
3   1634   1951   2401   1388    577    386   1166   1811   2076   1899   
4   2419  -3198  -5572  -3938  -4115  -3154  -1565    892    778   -682   

  Xac010 Xac011 Xac012 Xac013 Xac014 Xac015 Xac016 Xac017 Xac018 Xac019  \
0  -1618  -1746  -1859  -1791  -1672  -1342   -949   -935   -885   -288   
1  -1982   -972   -315    436   1119   1515   2283   2367   2309   1878   
2   -139     71     63    308    302    566    966   1521   1070   1725   
3   1276   1408   1345   1107    946   1231   1291   1668   1705   1753   
4  -1737  -2414  -2819  -2562  -2092  -2554  -2775  -2993  -3093  -2822   

  Xac020 Xac021 Xac022 Xac023 Xac024 Xac025 Xac026 Xac027 Xac028 Xac029  \
0   -606     72   1135    714    991   1497   1554   2632   2448   2319   
1   1583   1945   3393   4661   5485   5632   5366   5207   5794   7047   
2   2231   1796   2409   2627   2368   2297   2499   2205   2347   2019   
3   1849   2150   2169   2242   2173   1952   1760   2057   2453   1885   
4  -2752  -2856  -2588  -1899  -1460  -1243   -929   -530   -229   -529   

  Xac030 Xac031 Xac032 Xac033 Xac034 Xac035 Xac036 Xac037 Xac038 Xac039  \
0   2289   2145   2083   2292   1997   1550   1747   1686   1764   1643   
1   7180   7404   8474  10112  12364  14802  15772  16350  19127  24855   
2   1715   1432   1299   1258   1222   1366   1332   1821   2920   3836   
3   1996   1902   1733   1270   1009   1055   1121   1263   1760   2716   
4    749    939   1543   1944   1867   1376   1135   1050   1029   1074   

  Xac040 Xac041 Xac042 Xac043 Xac044 Xac045  Xac046 Xac047 Xac048 Xac049  \
0   1218   1228   1450   1729   1765   2302    2233   1916   2128   2795   
1  30946  30894  24159  17890  14852   6456  -13024  30257  30306   9019   
2   3545   2057   3113   2444    371   -629    -510   -937  -2226  -3962   
3   3245   2984   3093   2619    889    290    -146  -1822  -3402  -6456   
4    885    741    718    415    531    358     347    422     13     99   

   Xac050 Xac051 Xac052 Xac053 Xac054 Xac055 Xac056 Xac057 Xac058 Xac059  \
0    2686   1743   2024   1860   -668  -2069  -3099  -5204  -9375  -9698   
1   27775   6133   6599   5973     34   -931   5121   8565   1794  -3591   
2   -5294  -4063   2183  12087   9140   4755   3553   1866  -1085  -2116   
3  -10012  -6847   2973  12507   6460   4409   5911   2950  -1476  -2139   
4     974   1507   2195   2084    618    915   1800    504   -730  -1648   

  Xac060 Xac061 Xac062 Xac063 Xac064 Xac065 Xac066 Xac067 Xac068 Xac069  \
0  -4032   8704  10506   6388   4405   1555    102    832    928    749   
1  -6217  -3841   -641    -68   1157   1279   -461  -1708  -2288  -2407   
2  -3143  -4841  -4463    152   7001   5694   1480    240    901    745   
3  -1207  -1586    559   6600   6917   1824    422   2038   3207   3584   
4  -2896  -4688  -7196  -8486  -7070  -2195   7785   9199   3711   3674   

  Xac070 Xac071 Xac072 Xac073 Xac074 Xac075 Xac076 Xac077 Xac078 Xac079  \
0   3690   8169   3806   1072    595    322   -839  -1494  -2619  -2728   
1  -2831  -2993  -1838  -2142  -1861  -1838   -860   -242    214    310   
2    364  -2727  -4688  -5806  -5703  -2386   -264    268    975   1225   
3   2908   2574   2273    -71  -1999  -4061  -3955  -1605   -656    179   
4   3481    864   -676   -869   -993  -1058    798   5380   5590   1332   

  Xac080 Xac081 Xac082 Xac083 Xac084 Xac085 Xac086 Xac087 Xac088 Xac089  \
0  -1505   -780   -199    374    908   1080   1019   1141   1556   2620   
1     80    297    -77    886   1521   1872   2428   1974   2030   2123   
2   1492   1319   1498   2040   4042   5879   6430   6844   7482   7742   
3   1327   1231   1136   2224   2922   4510   5238   5566   5863   7421   
4    -9

# Transformar Dados Raw em Arquivo de Aúdio WAV

In [49]:
# Retorna um profile_ID aleatório
def getRandID():
    
    profile_ID = list(df_horizontal['profile_ID']) # Coloca todos os profile_ID do dataframe em uma lista.

    # Retorna aleatoriamente UM elemento da lista profile_ID, converte para um inteiro e armazena na variável Rand_ID.
    Rand_ID = int(random.choice(profile_ID)) 
    
    return Rand_ID

# Retorna o status da marcha respectivo ao profile_ID aleatório gerado.
def getRandGaitStatus(Rand_ID):

    # Rand_ID representa a LINHA, 1 representa a COLUNA de Gait_Status.
    Rand_GaitStatus = df_horizontal.iloc[Rand_ID-1, 1] # O -1 é para pegar a correta indexação do elemento na lista.
    
    return Rand_GaitStatus
    
    


In [14]:
# Função para gerar 3 arquivos .wav (eixo x, y e z dos dados do acelerômetro) de uma pessoa
# aleatória específica com marcha NORMAL.
    
# Esta função deverá retornar uma lista com 6 elementos no seguinte formato:
# wav_gait = [0001Xacc_N.wav,0001Yacc_N.wav, 0001Zacc_N.wav]
def get_NWavFiles():
    # Fs = 100 Hz
    # Duração 5 segundos
    
    Rand_ID = getRandID () # Captura um profile_ID aleatório.
    Rand_GaitStatus = getRandGaitStatus(Rand_ID) # Captura o estado da marcha relativo ao profile_ID aleatório.


    while (Rand_GaitStatus != 'N'):
        Rand_ID = getRandID ()
        Rand_GaitStatus = getRandGaitStatus(Rand_ID)
    

    if (Rand_GaitStatus == 'N'):

                            #################### Procedimento para Eixo X #################

        # Cria uma arquivo do tipo.wav para escrita com o nome fornecido.
        obj = wave.open('/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/Xacc{0}_{1}.wav'.format
                                                                                (Rand_ID, Rand_GaitStatus), 'wb')
        obj.setnchannels(1) #Set the number of channels. 1 for Mono 2 for stereo channels
        obj.setsampwidth(2) # Set the sample width to n bytes.
        obj.setframerate(100) # Set the frame rate to n. (Fs = 100 Hz)
        obj.setnframes(5) #Set the number of frames to n.

        # Retorna uma lista com todos os valores de aceleração do eixo X.
        Acc_AxisX= list(df_horizontal.loc[Rand_ID-1, 'Xac000':'Xac499'].values)

        # Transformar os dados raw em dados binários através da biblioteca struct.
        for value in Acc_AxisX:
           
            data = struct.pack('<h', value) # '<' representa um parâmetro de alinhamento e h significa short int.
                                            # Aqui short tem um tamanho de 2 bytes e abrange todo o range dos dados
                                            #coletados. '>' é o "numbers in little-endian C layout"

            obj.writeframesraw( data ) # Write audio frames, without correcting.

        obj.close() # Close the file if it was opened by wave.
        
                        #################### Procedimento para Eixo Y #################
            
        # Cria uma arquivo do tipo.wav para escrita com o nome fornecido.
        obj = wave.open('/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/Yacc{0}_{1}.wav'.format
                                                                                (Rand_ID, Rand_GaitStatus), 'wb')
        obj.setnchannels(1) #Set the number of channels. 1 for Mono 2 for stereo channels
        obj.setsampwidth(2) # Set the sample width to n bytes.
        obj.setframerate(100) # Set the frame rate to n. (Fs = 100 Hz)
        obj.setnframes(5) #Set the number of frames to n.

        # Retorna uma lista com todos os valores de aceleração do eixo X.
        Acc_AxisY= list(df_horizontal.loc[Rand_ID-1, 'Yac000':'Yac499'].values)

        # Transformar os dados raw em dados binários através da biblioteca struct.
        for value in Acc_AxisY:

            data = struct.pack('<h', value) # '<' representa um parâmetro de alinhamento e h significa short int.
                                            # Aqui short tem um tamanho de 2 bytes e abrange todo o range dos dados
                                            #coletados. '>' é o "numbers in little-endian C layout"

            obj.writeframesraw( data ) # Write audio frames, without correcting.

        obj.close() # Close the file if it was opened by wave.

                        #################### Procedimento para Eixo Z #################

        # Cria uma arquivo do tipo.wav para escrita com o nome fornecido.
        obj = wave.open('/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/Zacc{0}_{1}.wav'.format
                                                                                (Rand_ID, Rand_GaitStatus), 'wb')
        obj.setnchannels(1) #Set the number of channels. 1 for Mono 2 for stereo channels
        obj.setsampwidth(2) # Set the sample width to n bytes.
        obj.setframerate(100) # Set the frame rate to n. (Fs = 100 Hz)
        obj.setnframes(5) #Set the number of frames to n.

        # Retorna uma lista com todos os valores de aceleração do eixo X.
        Acc_AxisZ= list(df_horizontal.loc[Rand_ID-1, 'Zac000':'Zac499'].values)

        # Transformar os dados raw em dados binários através da biblioteca struct.
        for value in Acc_AxisZ:

            data = struct.pack('<h', value) # '<' representa um parâmetro de alinhamento e h significa short int.
                                            # Aqui short tem um tamanho de 2 bytes e abrange todo o range dos dados
                                            #coletados. '>' é o "numbers in little-endian C layout"

            obj.writeframesraw( data ) # Write audio frames, without correcting.

        obj.close() # Close the file if it was opened by wave.
        
        
        

        return ['Xacc{0}_{1}.wav'.format(Rand_ID, Rand_GaitStatus),
                'Yacc{0}_{1}.wav'.format(Rand_ID, Rand_GaitStatus),
                'Zacc{0}_{1}.wav'.format(Rand_ID, Rand_GaitStatus)]






    #Próximas implementações:
    # -> 1.Ajustar para que o normal e irregular se refiram ao mesmo indivíduo.
    # -> 2. As comparações de plots devem ser sempre com os mesmos eixos respectivos: x-x , y-y, z-z da aceleração.
    # -> 3. Função que me permita apagar todos os arquivos .wav gerados.


In [15]:
# Função para gerar 3 arquivos .wav (eixo x, y e z dos dados do acelerômetro) de uma pessoa
# aleatória específica com marcha IRREGULAR.
    
# Esta função deverá retornar uma lista com 6 elementos no seguinte formato:
# wav_gait = [0005Xacc_I.wav, 0005Yacc_I.wav, 0005Zacc_I.wav]
def get_IWavFiles():
    # Fs = 100 Hz
    # Duração 5 segundos
    
    Rand_ID = getRandID () # Captura um profile_ID aleatório.
    Rand_GaitStatus = getRandGaitStatus(Rand_ID) # Captura o estado da marcha relativo ao profile_ID aleatório.


    while (Rand_GaitStatus != 'I'):
        Rand_ID = getRandID ()
        Rand_GaitStatus = getRandGaitStatus(Rand_ID)
    

    if (Rand_GaitStatus == 'I'):

                            #################### Procedimento para Eixo X #################

        # Cria uma arquivo do tipo.wav para escrita com o nome fornecido.
        obj = wave.open('/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/Xacc{0}_{1}.wav'.format
                                                                                (Rand_ID, Rand_GaitStatus), 'wb')
        obj.setnchannels(1) #Set the number of channels. 1 for Mono 2 for stereo channels
        obj.setsampwidth(2) # Set the sample width to n bytes.
        obj.setframerate(100) # Set the frame rate to n. (Fs = 100 Hz)
        obj.setnframes(5) #Set the number of frames to n.

        # Retorna uma lista com todos os valores de aceleração do eixo X.
        Acc_AxisX= list(df_horizontal.loc[Rand_ID-1, 'Xac000':'Xac499'].values)

        # Transformar os dados raw em dados binários através da biblioteca struct.
        for value in Acc_AxisX:
           
            data = struct.pack('<h', value) # '<' representa um parâmetro de alinhamento e h significa short int.
                                            # Aqui short tem um tamanho de 2 bytes e abrange todo o range dos dados
                                            #coletados. '>' é o "numbers in little-endian C layout"

            obj.writeframesraw( data ) # Write audio frames, without correcting.

        obj.close() # Close the file if it was opened by wave.
        
                        #################### Procedimento para Eixo Y #################
            
        # Cria uma arquivo do tipo.wav para escrita com o nome fornecido.
        obj = wave.open('/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/Yacc{0}_{1}.wav'.format
                                                                                (Rand_ID, Rand_GaitStatus), 'wb')
        obj.setnchannels(1) #Set the number of channels. 1 for Mono 2 for stereo channels
        obj.setsampwidth(2) # Set the sample width to n bytes.
        obj.setframerate(100) # Set the frame rate to n. (Fs = 100 Hz)
        obj.setnframes(5) #Set the number of frames to n.

        # Retorna uma lista com todos os valores de aceleração do eixo X.
        Acc_AxisY= list(df_horizontal.loc[Rand_ID-1, 'Yac000':'Yac499'].values)

        # Transformar os dados raw em dados binários através da biblioteca struct.
        for value in Acc_AxisY:

            data = struct.pack('<h', value) # '<' representa um parâmetro de alinhamento e h significa short int.
                                            # Aqui short tem um tamanho de 2 bytes e abrange todo o range dos dados
                                            #coletados. '>' é o "numbers in little-endian C layout"

            obj.writeframesraw( data ) # Write audio frames, without correcting.

        obj.close() # Close the file if it was opened by wave.

                        #################### Procedimento para Eixo Z #################

        # Cria uma arquivo do tipo.wav para escrita com o nome fornecido.
        obj = wave.open('/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/Zacc{0}_{1}.wav'.format
                                                                                (Rand_ID, Rand_GaitStatus), 'wb')
        obj.setnchannels(1) #Set the number of channels. 1 for Mono 2 for stereo channels
        obj.setsampwidth(2) # Set the sample width to n bytes.
        obj.setframerate(100) # Set the frame rate to n. (Fs = 100 Hz)
        obj.setnframes(5) #Set the number of frames to n.

        # Retorna uma lista com todos os valores de aceleração do eixo X.
        Acc_AxisZ= list(df_horizontal.loc[Rand_ID-1, 'Zac000':'Zac499'].values)

        # Transformar os dados raw em dados binários através da biblioteca struct.
        for value in Acc_AxisZ:

            data = struct.pack('<h', value) # '<' representa um parâmetro de alinhamento e h significa short int.
                                            # Aqui short tem um tamanho de 2 bytes e abrange todo o range dos dados
                                            #coletados. '>' é o "numbers in little-endian C layout"

            obj.writeframesraw( data ) # Write audio frames, without correcting.

        obj.close() # Close the file if it was opened by wave.
        
        
        

        return ['Xacc{0}_{1}.wav'.format(Rand_ID, Rand_GaitStatus),
                'Yacc{0}_{1}.wav'.format(Rand_ID, Rand_GaitStatus),
                'Zacc{0}_{1}.wav'.format(Rand_ID, Rand_GaitStatus)]






    #Próximas implementações:
    # -> 1.Ajustar para que o normal e irregular se refiram ao mesmo indivíduo.
    # -> 2. As comparações de plots devem ser sempre com os mesmos eixos respectivos: x-x , y-y, z-z da aceleração.
    # -> 3. Função que me permita apagar todos os arquivos .wav gerados.

In [16]:
get_IWavFiles()

['Xacc83_I.wav', 'Yacc83_I.wav', 'Zacc83_I.wav']

# Ler algumas amostras de Aúdio

In [17]:
sr = 16000
e_file1 = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/gait_X.wav'

# 5 segundos de amostragem
y1, sr = librosa.load(e_file1, mono=True, sr=sr, offset=0, duration=5)

/home/trs/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


FileNotFoundError: [Errno 2] No such file or directory: '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/gait_X.wav'

## Escutar a amostra de Aúdio

In [ ]:
from IPython.display import Audio, IFrame, display

display(Audio(y1,rate=sr))

##### O aúdio acima representa a geração de um arquivo .wav a partir de dados raw do acelerômetro no eixo X. Foram tomadas 500 amostras em um intervalo de aproximadamente = 5 segundos para formar esse arquivo em formato de aúdio .wav

# Etapa X.x.x: Comparação entre as curvas de resposta


In [ ]:
P1WavAccN = get_NWavFiles()
P2WavAccN = get_NWavFiles()
    

P1WavAccI = get_IWavFiles()
P2WavAccI = get_IWavFiles()

sr = 16000

### Comparação da marcha normal entre dois indivíduos referentes à aceleração ao longo do eixo x.


In [ ]:
P1WavAccN_X = P1WavAccN[0]
P2WavAccN_X = P2WavAccN[0]


P1wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P1WavAccN_X)
P2wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P2WavAccN_X)

In [ ]:
# 5 segundos de amostragem
P1WavAccN_X, sr = librosa.load(P1wav_path, mono=True, sr=sr, offset=0, duration=5)
P2WavAccN_X, sr = librosa.load(P2wav_path, mono=True, sr=sr, offset=0, duration=5)


In [ ]:
librosa.display.waveplot(P1WavAccN_X,sr=sr,x_axis='time')


In [ ]:
librosa.display.waveplot(P2WavAccN_X,sr=sr,x_axis='time')


### Comparação da marcha normal entre dois indivíduos referentes à aceleração ao longo do eixo y.


In [ ]:
P1WavAccN_Y = P1WavAccN[1]
P2WavAccN_Y = P2WavAccN[1]


P1wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P1WavAccN_Y)
P2wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P2WavAccN_Y)

In [ ]:
# 5 segundos de amostragem
P1WavAccN_Y, sr = librosa.load(P1wav_path, mono=True, sr=sr, offset=0, duration=5)
P2WavAccN_Y, sr = librosa.load(P2wav_path, mono=True, sr=sr, offset=0, duration=5)

In [ ]:
librosa.display.waveplot(P1WavAccN_Y,sr=sr,x_axis='time')

In [ ]:
librosa.display.waveplot(P2WavAccN_Y,sr=sr,x_axis='time')


### Comparação da marcha normal entre dois indivíduos referentes à aceleração ao longo do eixo z.


In [ ]:
P1WavAccN_Z = P1WavAccN[2]
P2WavAccN_Z = P2WavAccN[2]


P1wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P1WavAccN_Z)
P2wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P2WavAccN_Z)
print(P1WavAccN_Z, P1wav_path)

In [ ]:
# 5 segundos de amostragem
P1WavAccN_Z, sr = librosa.load(P1wav_path, mono=True, sr=sr, offset=0, duration=5)
P2WavAccN_Z, sr = librosa.load(P2wav_path, mono=True, sr=sr, offset=0, duration=5)

In [ ]:
librosa.display.waveplot(P1WavAccN_Z,sr=sr,x_axis='time')

In [ ]:
librosa.display.waveplot(P2WavAccN_Z,sr=sr,x_axis='time')


### Comparação da marcha normal entre dois indivíduos referentes à aceleração ao longo do eixo x.


In [ ]:
P1WavAccI_X = P1WavAccI[0]
P2WavAccI_X = P2WavAccI[0]


P1wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P1WavAccI_X)
P2wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P2WavAccI_X)
print(P1WavAccI_X, P1wav_path,P2wav_path)

In [ ]:
# 5 segundos de amostragem
P1WavAccI_X, sr = librosa.load(P1wav_path, mono=True, sr=sr, offset=0, duration=5)
P2WavAccI_X, sr = librosa.load(P2wav_path, mono=True, sr=sr, offset=0, duration=5)

In [ ]:
librosa.display.waveplot(P1WavAccI_X,sr=sr,x_axis='time')

In [ ]:
librosa.display.waveplot(P2WavAccI_X,sr=sr,x_axis='time')


### Comparação da marcha normal entre dois indivíduos referentes à aceleração ao longo do eixo x.


In [ ]:
P1WavAccN_X = P1WavAccN[0]
P2WavAccI_X = P2WavAccI[0]


P1wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P1WavAccN_X)
P2wav_path = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/RandWavFiles/{}'.format(P2WavAccI_X)
print(P1wav_path,P2wav_path)

In [ ]:
# 5 segundos de amostragem
P1WavAccN_X, sr = librosa.load(P1wav_path, mono=True, sr=sr, offset=0, duration=5)
P2WavAccI_X, sr = librosa.load(P2wav_path, mono=True, sr=sr, offset=0, duration=5)

In [ ]:
librosa.display.waveplot(P1WavAccN_X,sr=sr,x_axis='time')

In [ ]:
librosa.display.waveplot(P2WavAccI_X,sr=sr,x_axis='time')


##### O plot acima ilustra  bem o que é possível observar pela audição: em intervalos de tempos parecidos, um pico ( uma vibração forte) é possível de se escutar.

## Etapa 4.1.2: Logmel Espectograma

In [ ]:
S1 = librosa.feature.melspectrogram(y=y1, sr=sr, n_mels=64)
D1 = librosa.power_to_db(S1, ref=np.max)
librosa.display.specshow(D1, x_axis = 'time', y_axis='mel')

# [Pré Processamento] Escala dos Dados


In [15]:
# StandardScaler: Essa técnica na prática ignora a forma de distribuição e transforma o dado para forma com média
# próxima de zero e um desvio padrão próximo a um, ou seja, assume que não temos valores discrepantes nos dados
# e normaliza tudo.
from sklearn.preprocessing import MinMaxScaler

In [16]:
# Objeto para fazer a re-escala dos dados. Aqui ainda é um "recipiente" para os dados.
scaler = MinMaxScaler();

In [17]:
# Os dados das features do dataframe df são reestruturados a partir do chamada do método fit no objeto scaler.
# Aqui ainda não ocorre a transformação, mas sim uma "preparação" para a transformação.
scaler.fit(df_pca)

MinMaxScaler()

In [18]:
# Aqui os dados das colunas passadas no scaler.fit(df) sofrem uma tansformação. Os dados  possuem escalas ajustadas
# de modo  a prepará-los para executar um modelo de ML. 
scaled_data = scaler.transform(df_pca)


# [Pré Processamento] Redução com PCA
#### Redução de Dimensionalidade aplicando PCA

In [19]:
# Cria um objeto que servirá de "recipiente" para os dados "scaled_data".
# Observa-se que o número de componentes principais setado foi  = 60 (10 para cada eixo. Total 6*10).

# Preciso coletar mais dados, pois o n_components máximo é igual ao mínimo entre min(n_samples, n_features)=30.
pca = PCA(n_components=130)

# n_components entre 30 - 40 melhora significamente o algoritmo SVM com escala MinMaxScaler;
# Os outros modelos não atendem bem nessas configurações. Acurácia: 69%

# O melhor modelo encontrado até agora foi com DecisionTree não passando pelo processo de escala dos dados e 
# com n_components = 3. Acurácia: 72% - 74%

# Att 18/11/2020 : Melhor modelo encontrado até agora foi com SVM passando pelo StandarScaler e n_components >= 15. 
# Acurácia: 74% - 78%

In [20]:
# Dimensiona o objeto "recipiente" pca para os dados da variável scaled_data
#pca.fit(df_pca)
pca.fit(scaled_data)

######## AJUSTAR PARA df_pca OU scaled_data ... TESTAR ############

PCA(n_components=130)

In [21]:
# Apply the rotation in dimensionality reduction by calling transform
#x_pca = pca.transform(df_pca) 
x_pca = pca.transform(scaled_data)

In [22]:
scaled_data.shape # Dados padronizados antes da transformação PCA

(178, 3000)

In [23]:
x_pca.shape # Dados padronizados apenas com 30 features após a transformação PCA

(178, 130)

In [24]:
# Dataframe que representa o dataset original através da redução de dimensionalidades de 3000 features para 30.
df_xPCA = pd.DataFrame(x_pca) 
df_xPCA.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129
0,-0.778985,-1.454881,-4.909609,-2.766266,-3.833052,2.072544,-0.024304,-1.087086,-2.485420,0.731375,-0.675249,2.055085,-0.551757,-0.559160,-1.567519,-1.356753,0.454415,-0.810326,2.641362,-1.625624,0.382299,1.532378,-0.581730,-0.251253,-0.315329,0.078435,-0.051851,-0.583934,0.031047,0.639879,-1.178604,-0.220899,-1.037517,0.564900,-0.100312,0.193299,-0.584033,-0.652101,0.724121,0.216057,0.147693,-0.651530,0.595078,1.017759,0.955033,-0.297647,0.930709,0.920489,0.130640,-1.341090,0.771651,0.493102,-0.189510,0.734270,0.201805,-0.030302,-0.886585,-0.175284,0.223166,1.180290,-0.374360,-0.483483,-0.556805,0.145758,-0.606617,0.100401,0.007278,-0.929463,1.095851,0.345300,-0.108058,0.039159,-0.902108,-0.348560,0.012489,-0.473726,-0.513744,0.127525,-0.131547,0.235106,0.269890,0.455582,-0.283254,0.950398,-0.383667,-0.419394,-0.364038,-0.505654,0.073028,0.223835,-0.768246,0.281013,-0.342641,-0.005398,-0.521085,-0.291159,-0.395115,0.207639,-0.256521,-0.008209,0.399935,-0.379193,-0.070972,0.285299,0.000164,0.277857,0.219177,-0.393111,0.382245,-0.218765,-0.344411,-0.129816,-0.221888,0.388209,0.127732,-0.208768,0.295862,-0.057907,-0.103220,0.041397,-0.200079,-0.236264,0.177075,-0.168761,0.093091,-0.117475,-0.223087,0.251479,0.102001,0.122799
1,-0.424328,1.037673,1.143349,1.836273,3.339592,-3.036805,1.962279,-0.840442,0.458756,1.711711,1.704260,2.817703,-3.939013,-3.865387,-0.529040,-1.441544,0.012672,-0.344754,1.266798,2.683275,-1.717590,1.690580,-0.688662,-1.006936,-1.084695,1.211626,1.231145,1.143807,-0.770002,-0.870649,0.997209,-0.839742,0.116786,0.547441,1.004366,-1.992455,0.944176,-0.065808,-0.270993,-0.559815,0.881004,-0.038802,-1.209272,1.817723,-1.222753,0.454202,0.264719,-0.353453,0.689762,-0.173446,0.493819,-0.512376,0.199921,0.147790,0.446968,-0.036570,0.089754,0.153752,1.062161,0.278142,0.174953,-0.359626,-1.161875,-0.122265,0.067778,0.235763,-0.363822,0.434213,-0.021401,0.152399,-0.227310,0.057260,0.566201,0.112607,-0.055241,0.050915,-0.359561,-0.118942,0.280374,0.597190,-0.220778,-0.072436,-0.044152,-0.076926,-0.309030,0.442397,-0.373450,-0.293024,-0.096174,-0.410684,0.023749,-0.296153,-0.125490,-0.157932,-0.154180,-0.231544,-0.167450,0.015512,-0.002944,-0.003587,0.086405,-0.148532,-0.147521,0.244366,-0.098122,-0.031934,0.127391,0.061490,0.097783,-0.103582,-0.185114,-0.139610,0.212121,-0.111864,0.127189,0.172440,-0.034681,-0.172841,0.060551,-0.038355,-0.114144,-0.144535,-0.068410,0.134886,-0.077465,-0.218524,0.090516,0.020555,-0.179820,-0.079273
2,-0.422665,-1.579054,-4.245875,-2.197445,-3.153832,2.984608,-1.510633,3.567349,2.960844,0.980489,2.205778,2.010772,-2.314335,-1.939159,-0.715651,-0.248679,-1.717433,-1.338986,-0.590586,0.132523,1.179503,-1.144299,-0.748361,-0.925607,0.027788,-0.468355,-1.358656,-1.124202,0.663860,-1.201169,0.033763,0.579406,0.341842,-1.329098,0.733680,0.168282,0.637691,0.299529,-0.347784,0.219923,-0.491960,-0.012091,0.411802,-0.185392,0.111142,0.295923,0.300214,-0.267924,-0.345722,0.497085,-0.061115,-0.178698,-0.329738,-0.746184,-0.101080,0.357978,-0.009062,-0.587884,0.255813,-0.234692,0.531774,0.100793,0.034198,0.122135,-0.023780,0.524578,0.190650,-0.120266,-0.076000,0.084052,0.307240,0.664351,0.178441,-0.342254,0.201663,0.064700,-0.349204,0.089579,-0.034261,0.189702,0.007178,0.153502,-0.386825,-0.212312,0.152236,-0.190850,-0.262496,-0.070349,-0.158878,-0.506145,-0.009733,-0.166176,-0.522414,0.035768,0.534357,0.197335,0.237217,-0.090197,-0.432021,-0.097606,-0.033685,0.218473,0.141584,-0.337537,-0.454423,0.345084,-0.153821,0.025246,0.486974,0.485553,-0.148925,-0.424256,-0.436300,-0.087115,0.546555,-0.8

## Plots

In [25]:
#sns.heatmap()
#sns.pairplot(df_xPCA)
#sns.pairplot(df_dummies,x_vars=['Xac000', 'Xac001'],y_vars='Gait_Status_N')
#sns.distplot(df_dummies['Gait_Status_N'])
#sns.pairplot(df_dummies)


# Modelo de Regressão Logística

In [26]:
#y_target = df_dummies.loc[:,['Gait_Status_N']].values
y = df_dummies['Gait_Status_N'] # Feature target com apenas duas classes : 1 ( Normal) ou 0 (Irregular)
#X = df_dummies.iloc[:,1:3001]
X = x_pca # Features após a redução de dimensionalidade de 3000 para 30. 
#X = df_pca.loc[:,'Xac499':'RotY499']



In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
lm = LogisticRegression()

In [31]:
lm.fit(X_train,y_train)

LogisticRegression()

In [32]:
predictions = lm.predict( X_test)

In [33]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))


# print(classification_report(y_test,predictions)) #Após aplicação do PCA, acurácia subiu de 0.33 para 0.56
#                                                  #Após o dobrou do número de dataset csv files, aumentou de 0.56
#                                                  # para 0.8
# # random_state para 89% = 4

[[20 11]
 [ 8 15]]


              precision    recall  f1-score   support

           0       0.71      0.65      0.68        31
           1       0.58      0.65      0.61        23

    accuracy                           0.65        54
   macro avg       0.65      0.65      0.65        54
weighted avg       0.66      0.65      0.65        54



In [34]:
from sklearn.model_selection import cross_val_score
model = LogisticRegression()
scores = cross_val_score(model, X, y, cv=10, scoring= "accuracy")
scores

array([0.27777778, 0.5       , 0.77777778, 0.38888889, 0.44444444,
       0.61111111, 0.77777778, 0.66666667, 0.52941176, 0.70588235])

In [35]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.57 (+/- 0.32)


In [36]:
import joblib

In [41]:
# save the model to disk
filename = '/home/trs/Gait_Project/GaitDataset/dataset_v1.1/Models/LogisticRegression/finalized_model.sav'
joblib.dump(lm, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

NameError: name 'joblib' is not defined

# SVM MACHINE LEARNING MODEL

In [186]:
#y_target = df_dummies.loc[:,['Gait_Status_N']].values
y = df_dummies['Gait_Status_N'] # Feature target com apenas duas classes : 1 ( Normal) ou 0 (Irregular)
#X = df_dummies.iloc[:,1:3001]
X = x_pca # Features após a redução de dimensionalidade de 3000 para 30. 
#X = df_pca.loc[:,'RotZ000':'RotZ499']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [192]:
from sklearn.svm import SVC

In [193]:
model = SVC()

In [194]:
model.fit(X_train,y_train)

SVC()

In [195]:
predictions = model.predict(X_test)

In [196]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[23  8]
 [ 5 18]]


              precision    recall  f1-score   support

           0       0.82      0.74      0.78        31
           1       0.69      0.78      0.73        23

    accuracy                           0.76        54
   macro avg       0.76      0.76      0.76        54
weighted avg       0.77      0.76      0.76        54



In [216]:
from sklearn.model_selection import cross_val_score

# Utilizando as métricas do cross_val_score
scores = cross_val_score(SVC(), X, y, cv=LeaveOneOut(), scoring= "accuracy")
scores


array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

In [217]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.79 (+/- 0.81)


In [218]:
 
from sklearn.model_selection import cross_validate, LeaveOneOut

# Utilizando as métricas do cross_validate
cross_validate(SVC(),X,y,cv=LeaveOneOut(), scoring = 'accuracy')['test_score'].mean()

0.7921348314606742

In [149]:
from sklearn.model_selection import GridSearchCV

In [166]:
#param_grid = {'C':[0.0001,0.001,0.1,1,10,100,1000, 100000], 'gamma':[1,0.1,0.01,0.001,0.0001,0.00001]}

# # defining parameter range 
# param_grid = {'C': [0.1, 1, 10, 100, 1000],  
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}  
  
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

In [231]:
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 

In [232]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.600, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.640, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.520, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.400, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.667, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.560, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ................. C=10, kernel=linear, score=0.520, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.480, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.625, total=   0.0s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.560, total=   0.7s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.640, total=   0.5s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.520, total=   1.9s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.520, total=   0.5s
[CV] C=100, kernel=linear ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   38.2s finished


GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             verbose=3)

In [233]:
grid.best_params_

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

In [234]:
grid.best_estimator_

SVC(C=100, gamma=0.001)

In [235]:
grid_predictions = grid.predict(X_test)

In [236]:
print(confusion_matrix(y_test,grid_predictions))
print('\n')
print(classification_report(y_test,grid_predictions))

[[24  7]
 [ 3 20]]


              precision    recall  f1-score   support

           0       0.89      0.77      0.83        31
           1       0.74      0.87      0.80        23

    accuracy                           0.81        54
   macro avg       0.81      0.82      0.81        54
weighted avg       0.83      0.81      0.82        54



In [238]:
scores = cross_val_score(grid, X, y, cv=2, scoring= "accuracy")
scores

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.667, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.833, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.889, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.833, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.882, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.667, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ................. C=10, kernel=linear, score=0.833, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.529, total=   0.0s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.500, total=   0.0s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.611, total=   0.4s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.500, total=   0.0s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.889, total=   0.0s
[CV] C=100, kernel=linear ............................................
[CV] ................ C=100, kernel=linear, score=0.529, total=   0.2s
[CV] C=1000, kernel=linear ...........................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    5.3s finished


array([0.46067416, 0.59550562])

In [239]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.53 (+/- 0.13)


# Decision Tree Algorithm

In [416]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion =  'entropy')

In [417]:
#y_target = df_dummies.loc[:,['Gait_Status_N']].values
y = df_dummies['Gait_Status_N'] # Feature target com apenas duas classes : 1 ( Normal) ou 0 (Irregular)
#X = df_dummies.iloc[:,1:3001]
X = x_pca # Features após a redução de dimensionalidade de 3000 para 30. 
#X = df_pca.loc[:,'Xac000':'RotX499']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [418]:
model.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [419]:
predictions = model.predict(X_test)

In [420]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[17 14]
 [ 9 14]]


              precision    recall  f1-score   support

           0       0.65      0.55      0.60        31
           1       0.50      0.61      0.55        23

    accuracy                           0.57        54
   macro avg       0.58      0.58      0.57        54
weighted avg       0.59      0.57      0.58        54



In [421]:

model.score(X_test,y_test)

0.5740740740740741

In [422]:
from sklearn.model_selection import cross_val_score
model = DecisionTreeClassifier()
# model.fit(X_train, y_train)
scores = cross_val_score(model, X, y, cv=15, scoring= "accuracy")
scores

array([0.66666667, 0.25      , 0.41666667, 0.41666667, 0.5       ,
       0.5       , 0.66666667, 0.66666667, 0.5       , 0.58333333,
       0.75      , 0.75      , 0.83333333, 0.36363636, 0.90909091])

In [423]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.58 (+/- 0.36)


In [104]:
criterion = ['gini', 'entropy']
max_depth = [2,4,6,8,10,12]

In [105]:
parameters = dict(criterion=criterion)
parameters

{'criterion': ['gini', 'entropy']}

In [106]:
grid = GridSearchCV(DecisionTreeClassifier(), parameters, refit = True, verbose = 3) 

NameError: name 'GridSearchCV' is not defined

In [70]:
grid.fit(X_train,y_train)

NameError: name 'grid' is not defined

In [71]:
grid.best_params_

NameError: name 'grid' is not defined

In [72]:
grid.best_estimator_

NameError: name 'grid' is not defined

In [73]:
grid_predictions = grid.predict(X_test)

NameError: name 'grid' is not defined

In [1385]:
print(confusion_matrix(y_test,grid_predictions))
print('\n')
print(classification_report(y_test,grid_predictions))

[[ 8  5]
 [10  7]]


              precision    recall  f1-score   support

           0       0.44      0.62      0.52        13
           1       0.58      0.41      0.48        17

    accuracy                           0.50        30
   macro avg       0.51      0.51      0.50        30
weighted avg       0.52      0.50      0.50        30



In [1386]:
from sklearn.model_selection import cross_val_score
model = DecisionTreeClassifier()
# model.fit(X_train, y_train)
scores = cross_val_score(grid, X, y, cv=30, scoring= "accuracy")
scores

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.550, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.579, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.579, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.632, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.526, total=   0.0s
[CV] criterion=entropy ...............................................
[CV] ................... criterion=entropy, score=0.550, total=   0.0s
[CV] criterion=entropy ...............................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concur

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.500, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.650, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.526, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.421, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.421, total=   0.0s
[CV] criterion=entropy ...............................................
[CV] ................... criterion=entropy, score=0.650, total=   0.0s
[CV] criterion=entropy ...............................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concur

[CV] ................... criterion=entropy, score=0.368, total=   0.0s
[CV] criterion=entropy ...............................................
[CV] ................... criterion=entropy, score=0.421, total=   0.0s
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.450, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.750, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.526, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.368, total=   0.0s
[CV] criterion=gini ..................................................
[CV] ...................... criterion=gini, score=0.684, total=   0.0s
[CV] criterion=en

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finish

array([0.25      , 0.5       , 0.75      , 0.5       , 0.5       ,
       0.75      , 0.75      , 0.5       , 0.5       , 0.5       ,
       0.33333333, 1.        , 0.33333333, 0.33333333, 0.33333333,
       1.        , 1.        , 0.33333333, 0.33333333, 0.66666667,
       0.33333333, 1.        , 0.33333333, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 1.        , 1.        , 0.33333333])

In [1387]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.59 (+/- 0.50)


In [1409]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification( n_features=3
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)


SyntaxError: invalid syntax (<ipython-input-1409-99b62b9fa7e9>, line 4)

In [1410]:
predictions = clf.predict(X_test)

ValueError: Number of features of the model must match the input. Model n_features is 20 and input n_features is 3 

In [ ]:
print(confusion_matrix(y_test,grid_predictions))
print('\n')
print(classification_report(y_test,grid_predictions))